# Joblib + Ray
(see [bottom](https://joblib.readthedocs.io/en/latest/parallel.html))

Short recipe to setup a Ray cluster on SLURM and interact with it using Joblib.

- start a "head" with the command ```ray start --head --port=6379 --num-cpus 1 --block```
- submit as many ```submit_worker.sub``` as many workers we want, taking care of changing the parameter inside according to the ip address of the head
- run the cells

In [ ]:
import ray
from ray.util.joblib import register_ray

In [ ]:
from joblib import parallel_backend, Parallel, delayed

In [ ]:
from tasks import dummy_function

In [ ]:
ray.init(address='192.33.123.23:6379')

In [ ]:
register_ray()

In [ ]:
nodes = ray.nodes()
alive_nodes = [node for node in nodes if node['Alive'] is True]

print('Nodes: {}\nAlive: {}'.format(len(nodes), len(alive_nodes)))

In [ ]:
%%time
# Code incapsulated in parallel_backend is scheduled and waited for
# As can be seen in the dashboard, this is correctly spread over the cluster

with parallel_backend('ray'):
    results = Parallel()(delayed(dummy_function)(n) for n in range(6))

In [ ]:
results

In [ ]:
%%time
# This isn't parallelized over the cluster

results = Parallel()(delayed(dummy_function)(n) for n in range(6))

In [ ]:
%%time
# This is correctly parallelized LOCALLY

results = Parallel(n_jobs=6)(delayed(dummy_function)(n) for n in range(6))

In [ ]:
def nested_function(arg):
    with parallel_backend('ray'):
        results = Parallel()(delayed(dummy_function)(n) for n in range(arg))
    return results

def simple_nested_function(arg):
    return Parallel(n_jobs=arg)(delayed(dummy_function)(n) for n in range(arg))

def simple_nested_function_no_arg(arg):
    return Parallel()(delayed(dummy_function)(n) for n in range(arg))

In [ ]:
%%time

with parallel_backend('ray'):
    results = Parallel()(delayed(simple_nested_function_no_arg)(n) for n in [4, 2])

In [ ]:
results